## 1) Identify the Three Structural Layers in ATT&CK STIX

ATT&CK, when expressed as STIX, has **three distinct structural layers**.
Understanding these layers is mandatory before building a knowledge graph.

### A. Objects (Nodes)

In STIX, **objects** are all items where:

- `type != "relationship"`

Common ATT&CK object types include:

- `attack-pattern`  
  - Techniques and sub-techniques
- `x-mitre-tactic`
- `intrusion-set`
- `malware`
- `tool`
- `campaign`
- `course-of-action` (mitigations; dataset/version dependent)
- `x-mitre-data-source`
- `x-mitre-data-component`
- `identity`, `marking-definition` (metadata)

**Analysis questions:**

- What object types exist in this dataset?
- Which object types are *core* to my use case?
- Which object types are metadata-only?
- Which fields are always present vs optional?
- Which identifiers are canonical (STIX ID vs external ID)?

### B. Relationships (Edges)

Relationships are STIX objects where:

- `type == "relationship"`

Each relationship contains:

- `relationship_type`
- `source_ref`
- `target_ref`

Common ATT&CK relationship types:

- `uses`
- `mitigates`
- `subtechnique-of`
- `revoked-by`
- `detects`
- `associated-with`

**Analysis questions:**

- What relationship types exist?
- Which ones matter for my graph?
- What *kinds* of objects do they connect?
- Are some relationships version-specific?

### C. Embedded Edges: Kill Chain Phases (Technique → Tactic)

ATT&CK does **not** model technique → tactic as a STIX relationship.

Instead, techniques (`attack-pattern`) embed:

```json
kill_chain_phases: [
  { kill_chain_name, phase_name }
]


## 2) Extracting the ATT&CK Schema from STIX

This section defines *what schema information you must extract* from the dataset.

### Step 1 — Node Taxonomy

Goal: understand the **shape of node objects**.

You must answer:

- What STIX object types exist?
- How many objects per type?
- Which fields appear consistently?
- Which fields are sparse or optional?

**Expected outputs:**

- `type → count`
- For each type:
  - set of observed fields
  - % presence per field
- Canonical identifier rules per type

### Step 2 — Edge Taxonomy

Goal: understand **valid structural connections**.

You must answer:

- What relationship types exist?
- What `(source_type, relationship_type, target_type)` combinations are valid?

**Expected output:**

A table like:

```text
(src_type, rel_type, dst_type) → count

## 3) — Hierarchies and Special Structure

ATT&CK encodes meaning beyond simple nodes and edges.

#### 3.1 Technique ↔ Sub-Technique

Facts:

- Both are `attack-pattern`
- Sub-techniques may be indicated by:
  - `x_mitre_is_subtechnique: true`
  - `subtechnique-of` relationship

**Analysis questions:**

- Which signal is authoritative?
- Do both always exist?
- How do you enforce parent/child constraints?

#### 3.2 Technique ↔ Tactic

Facts:

- Implemented via `kill_chain_phases`
- Not represented as STIX relationships

**Analysis questions:**

- Do you normalize this into explicit edges?
- Do you allow multiple tactics per technique?

#### 3.3 Lifecycle: Revocation and Deprecation

Objects may be:

- Revoked
- Deprecated

Signals include:

- `revoked: true`
- `x_mitre_deprecated: true`
- `revoked-by` relationships

**Analysis questions:**

- Do you drop revoked objects?
- Do you keep them with flags?
- How do you handle edges pointing to revoked nodes?

## 4) Minimal Questions That Prove ATT&CK Understanding

If your analysis can answer these, you understand the framework.

1. How many STIX object types exist, and which are core?
2. What are the most common relationship types?
3. What object types do those relationships connect?
4. How are tactics connected to techniques?
5. How do sub-techniques work structurally?
6. How do you detect sub-techniques reliably?
7. How do you handle revoked and deprecated objects?
8. Where do external IDs (e.g., `T1059`) live?
9. How do you map STIX IDs to ATT&CK IDs?

## 5) ATT&CK Schema Report
- Nodes types summary (counts)
- Edge type summary (counts)
- Valid triples (src_type, rel_type, dst_type)
- Special embedded edges (kill chain phases)
- Lifecycle rules (revoked/deprecated)

## 6) Concrete “start with ATT&CK” checklist (no fluff)
- Get the STIX bundle JSON (Enterprise ATT&CK).
- Count object types.
- Count relationship types.
- Build triple counts (src_type, rel_type, dst_type).
- Extract technique→tactic edges from kill_chain_phases.
- Extract technique→subtechnique edges.
- Decide lifecycle handling (revoked/deprecated).
- Decide your “core graph” vs “metadata-only” objects.